# Importações

In [1]:
import re
import spacy
import pickle
import os
import pandas as pd
import csv
import glob
import csv
from spacy.training import offsets_to_biluo_tags
from spacy.training import biluo_tags_to_spans
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_offsets, biluo_tags_to_spans
import itertools
from sklearn.model_selection import train_test_split
import sys
import matplotlib.pyplot as plt

# importações para o bloco de código do treinamento do modelo de NER
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.tokens import Doc
from spacy.training import Example
import warnings
import datetime as date
from datetime import datetime
import time
from spacy.scorer import Scorer
from sklearn.model_selection import cross_val_score

### Carregando os Dados
Na marcação IOB o prefixo B- antes de uma tag indica que a tag é o início de um pedaço, e um prefixo I- antes de uma tag indica que a tag está dentro de um pedaço. A tag B é usada somente quando uma tag é seguida por uma tag do mesmo tipo sem tokens O entre elas. Uma tag O indica que um token não pertence a nenhuma entidade/pedaço.
O dataset possui 5 atributos: Palavras (são os tokens que frmam a base de dados rotulada manualmente); Rotulo (são as categorias de entidades nomeadas seguidas pelo prefixo do tipo de marcação IOB; Sentenca (um número que represeta a sentença a que determinado token pertence); Inicio (número que representa o inicio de um token dentro de sua respctiva sentença); Fim (número que representa o fim de um token dentro de sua respctiva sentença).

In [10]:
#Dados de treinamento rotulado pelo algorimto de rotulação automatica
dataset_treinamento = []
with open("Resultado_com_10_porcento.txt", "r", encoding='utf-8') as file:
    for item in file:
        dataset_treinamento.append(item)
dataset_treinamento
#print("Tamanho do dataset de treinamento:", len(dados_treinamento)

["('NOME DA CACHAÇA: Porto Estrela Ouro 1 Litro', {'entities': [(0, 4, ''), (5, 7, ''), (8, 15, ''), (15, 16, ''), (17, 22, ''), (23, 30, ''), (31, 35, ''), (36, 43, 'VOLUME')]})\n",
 "('PREÇO: R$ 150,00', {'entities': [(0, 5, ''), (5, 6, ''), (7, 9, ''), (10, 13, ''), (13, 14, ''), (14, 16, '')]})\n",
 "('DESCRIÇÃO DA CACHAÇA: A Cachaça Porto Estrela é produzida na cidade de Pedras de Maria da Cruz , Minas Gerais, onde é engarrafada em vidro com tampa de rosca.', {'entities': [(0, 9, ''), (10, 12, ''), (13, 20, ''), (20, 21, ''), (22, 23, ''), (24, 31, ''), (32, 37, ''), (38, 45, ''), (46, 47, ''), (48, 57, ''), (58, 60, ''), (61, 67, ''), (68, 70, ''), (71, 77, ''), (78, 80, ''), (81, 86, ''), (87, 89, ''), (90, 94, ''), (95, 96, ''), (97, 102, ''), (103, 109, ''), (109, 110, ''), (111, 115, ''), (116, 117, ''), (118, 129, ''), (130, 132, ''), (133, 138, ''), (139, 142, ''), (143, 148, ''), (149, 151, ''), (152, 157, ''), (157, 158, '')]})\n",
 "('Cachaça de grande suavidade.', {'ent

In [12]:
#Carrega o dataset IOB, ou seja, os tokens são rotulados com tags do tipo marcação IOB.
dataset_teste = pd.read_csv("dataset_rotulado_teste.csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim','Documento'])
print("Tamanho do dataset de teste",len(dataset_teste))
dataset_teste

Tamanho do dataset de teste 53639


,Palavras,Rotulo,Sentenca,Inicio,Fim,Documento
0,NOME,O,1,0,4,1
1,DA,O,1,5,7,1
2,CACHAÇA,O,1,8,15,1
3,:,O,1,15,16,1
4,Cachaça,O,1,17,24,1
...,...,...,...,...,...,...
53634,Destilados,O,13618,53,63,999
53635,do,O,13618,64,66,999
53636,Brasil,B-NOME_LOCAL,13618,67,73,999
53637,2019,B-TEMPO,13618,74,78,999


In [20]:
#Carrega o dataset utilizada anteriormente na rotulação manual das categorias de entidades nomeadas, sem os IDs que identificam os documentos
#Esse arquivo é utilizado para extrairmos as setenças a que cada token rotulado no dataset pertence. 
#Essa informação é utilizada no código de conversão do dados no formato de marcação IOB para o formato compreendido pelo spaCy.
with open("dataset_para_rotulacao_manual_versao_2.txt", "r", encoding='utf-8') as file:
    textos = file.read().splitlines()
print("Quantida de de sentenças no dataSet utilizado na rotulação manual:", len(textos))

Quantida de de sentenças no dataSet utilizado na rotulação manual: 13628


### Convertendo os dados do formato IOB para o formato reconhecido pelo spaCy

Dados de treinamento

In [21]:
sMarker = 0 #Marcador que referência a qual setença a entidade pertence, ou seja, serve pra pegar o texto da referencia dos tokens no dataset
s = textos[sMarker]
dataSetEntradaTreinamento = []
entities = []
for index, row in dataset_treinamento.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            dataSetEntradaTreinamento.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]
            
    if(row['Rotulo'][0] == 'O'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if index == dataset_treinamento.index[-1]: # Ultimo elemento
        if entities:
            dataSetEntradaTreinamento.append((s, {"entities": [tuple(e) for e in entities]}))
        entities = [] 

print('Quantidade de sentenças do dataset de treinamento:', len(dataSetEntradaTreinamento))

Quantidade de sentenças do dataset de treinamento: 9454


Dados de teste

In [22]:
sMarker = 0 #Marcador que referência a qual setença a entidade pertence, ou seja, serve pra pegar o texto da referencia dos tokens no dataset
s = textos[sMarker]
dataSetEntradaTeste = []
entities = []
for index, row in dataset_teste.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            dataSetEntradaTeste.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]
            
    if(row['Rotulo'][0] == 'O'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
    if index == dataset_teste.index[-1]: # Ultimo elemento
        if entities:
            dataSetEntradaTeste.append((s, {"entities": [tuple(e) for e in entities]}))
        entities = [] 

print('Quantidade de sentenças do dataset de teste:', len(dataSetEntradaTeste))

Quantidade de sentenças do dataset de teste: 4174


## Importando modelo

In [23]:
# Carregando um modelo do spaCy em portguês previamente treinando
nlp = spacy.load('pt_core_news_sm')
print ("Modelo carregado '% s'"% nlp)

##carregando um modelo em braco
#nlp = spacy.blank('pt')
#nlp.vocab.vectors.name = 'spacy_pretrained_vectors'
#print ("Modelo carregado '% s'"% nlp)
#ner = nlp.create_pipe('ner')
#nlp.add_pipe('ner')

Modelo carregado '<spacy.lang.pt.Portuguese object at 0x000002ADB72ADFA0>'


## Customizando o Tokenizador para o modelo

In [25]:
from spacy.symbols import ORTH
from spacy.tokenizer import Tokenizer
from spacy.lang.char_classes import LIST_PUNCT, LIST_ELLIPSES, LIST_QUOTES, LIST_CURRENCY, LIST_HYPHENS
from spacy.lang.char_classes import LIST_ICONS, HYPHENS, CURRENCY
from spacy.lang.char_classes import CONCAT_QUOTES, ALPHA_LOWER, ALPHA_UPPER, ALPHA

# Essa função cria um objeto customizado da classe Tokenizer do spacy, para alinhamento dos tokens com a rotulações em alguns casos especiais por exemplo: '-' e '$'.
def custom_tokenizer(nlp):    
    infixes = (
        spacy.lang.char_classes.LIST_ELLIPSES
        + spacy.lang.char_classes.LIST_ICONS
        + ["\.", "\/", ":", "\(", "\)"]
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[\.])[\.](?=[\.])",
            r"(?<=[{a}{q}])\.(?=[{a}{q}])".format(
                a = ALPHA, q = CONCAT_QUOTES),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}])(?:{h}|\/)(?=[{a}])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[{a}])(?:{h}|\/)(?=[0-9])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[0-9])(?:{h}|\/)(?=[{a}])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[0-9])(?:{h}|\/|,|\.)(?=[0-9])".format(a=ALPHA, h=LIST_HYPHENS),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
            r"(?<=[{a}\"]),(?=[0-9{a}])".format(a=ALPHA),
        ]
    )
    list_quotes = ["\\'", '"', '`',
                     '‘', '´', '’', '‚', ',',
                     '„', '»', '«', '「', '」',
                     '『', '』', '（', '）', '〔',
                     '〕', '【', '】', '《', '》',
                     '〈', '〉']
    suffixes = (
        LIST_PUNCT
        + LIST_HYPHENS
        + list_quotes
        + LIST_ICONS
        + [ "\/", ":", "\(", "\)"]
        + ["'s", "'S", "’s", "’S", "—", "–"]
        + [
            r"(?<=[0-9])\+",
            r"(?<=[0-9])(?:{c})".format(c=CURRENCY),
            r"(?<=[0-9a-z(?:{q})])\.".format(
            al=ALPHA_LOWER, q=CONCAT_QUOTES),
            r"(?<=[{au}])\.".format(au=ALPHA_UPPER),
        ] 
    )
    
    prefixes = (
        LIST_PUNCT
        + [ "\/", ":", "\(", "\)"]
        + LIST_HYPHENS
        + list_quotes
        + LIST_CURRENCY
        + LIST_ICONS
    )

    infix_re = spacy.util.compile_infix_regex(infixes)
    suffix_re = spacy.util.compile_suffix_regex(suffixes)
    prefix_re = spacy.util.compile_prefix_regex(prefixes)
    custom = Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=None)
    special_case = [{ORTH: "."}, {ORTH: "."}, {ORTH: "."}]        # Adicionando uma regra especial
    custom.add_special_case("...", special_case)
    return custom


In [27]:
# atribuindo o tokenizador customizado ao modelo nlp
nlp.tokenizer = custom_tokenizer(nlp)

### Configurando parâmetros e dados para o treinamento do modelo

In [28]:
# Obtendo o componente do pipeline (ner) para trabalhar com reconhecimentos de entidades nomeadas
ner = nlp.get_pipe('ner')
# Adicionando as categorias de entidade nomeada ao pipeline 'ner' manualmente
ner.add_label('CARACTERISTICA_SENSORIAL_AROMA')
ner.add_label('CARACTERISTICA_SENSORIAL_CONSISTÊNCIA')
ner.add_label('CARACTERISTICA_SENSORIAL_SABOR')
ner.add_label('CARACTERISTICA_SENSORIAL_COR')
ner.add_label('RECIPIENTE_ARMAZENAMENTO')
ner.add_label('EQUIPAMENTO_DESTILACAO')
ner.add_label('CLASSIFICACAO_BEBIDA')
ner.add_label('TEMPO_ARMAZENAMENTO')
ner.add_label('GRADUACAO_ALCOOLICA')
ner.add_label('TIPO_MADEIRA')
ner.add_label('NOME_BEBIDA')
ner.add_label('VOLUME')
ner.add_label('NOME_LOCAL')
ner.add_label('NOME_ORGANIZACAO')
ner.add_label('NOME_PESSOA')
ner.add_label('PRECO')
ner.add_label('TEMPO')

#print("Categorias de entidade que o modelo contém:", '\n\n', ner.label_data)
# outra maneira mais rápida de adicionar as categorias de entidades ao 'ner'
#ner = nlp.get_pipe('ner')
#for texto, annotations in TRAIN_DATA:
    #for ent in annotations.get("entities"):
        #ner.add_label(ent[2])     
        #print(ent[2])
#print("Categorias de entidade que o modelo contém:", '\n\n', ner.label_data) #verificando as categorias de entidades nomeadas contidas no pipeline

1

In [29]:
# Obtendo os nomes dos componentes que queremos desativar durante o treinamento
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#Abaixo temos outra maneira de fazer a mesma cois, mas com duas linhas de comando
#pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
#unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [30]:
#Tranforma os dados do conjunto de treino no formato entendido pelo modelo ner
examples_train = []
for text, annots in dataSetEntradaTreinamento:
    examples_train.append(Example.from_dict(nlp.make_doc(text), annots))

### Treinando o modelo de reconhecimento de entidades nomeadas
O código abaixo realiza o treinamento do modelo de reconhecimento de entidades nomeadas

In [ ]:
#VTreinamento do modelo ner com novas categorias de entidades nomeadas
import decimal
optimizar = nlp.create_optimizer() #quando utilizamos o modelo já treinado do spaCy
#optimizar = nlp.begin_training() #quando é criado um modelo nlp vazio
epochsSize = 50
batchSize = 64
drop_size = 0.30
losses = {}
with nlp.disable_pipes(*unaffected_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy') 
    start_epoch = time.time()
    for epoch in range(epochsSize):
        random.shuffle(examples_train)
        #count = 0
        for batch in minibatch(examples_train, size=batchSize):
            nlp.update(batch, losses=losses, drop=drop_size, sgd=optimizar) 
            and_epoch = time.time()            
            #count+=1
            #valorPerda = losses['ner'] # valorPerda = decimal.Decimal(losses['ner'])
            #print("Perda acumulada ({}) para o batch ({})" .format(valorPerda,count,"\n"))
            tempoParaTreinarModelo = and_epoch-start_epoch 
            #metricas = nlp.evaluate(examples_validation)
            #print('Métricas de validação do modelo para o batch {}'.format(count),'\n')
            #print(metricas,'\n')

        #print("Perda acumulada ({}) para a epóca ({} de {})" .format(losses, epoch + 1, epochsSize),'\n') 
    print("Perda acumulada do modelo ({})" .format(losses),"\n")
    print('Tempo gasto para treinar o modelo em segundos -->', tempoParaTreinarModelo,'\n')
    #metricas=nlp.evaluate(examples_validation)
    #print('Métricas Final de validação do modelo com {} épocas e {} batchs'.format(epochsSize),'\n') 
    #print(metricas,'\n') 
    #scorer = Scorer()
    #result=scorer.score(examples)
    #print('SCORER POR BATC:', result)

### Comparação Rotulação Manual versus Rotulação do Modelo

In [ ]:
#Tranforma os dados do conjunto de teste no formato entendido pelo modelo ner
examples_test = []
for text, annots in dataSetEntradaTeste:
    examples_test.append(Example.from_dict(nlp.make_doc(text), annots))

In [247]:
from collections import Counter
import collections
# Compara item por item como o modelo classificou as sentenças e como elas foram classificadas manual (ponto de referencia).
#nlp = spacy.load(output_dir)
examples = []
comparacao= []
lista_rotulacaomanual = []
lista_rotulacaoautomatica = []
lista_merge_acertos = []
lista_merge_erros = []
count = 0
for text, annots in dataSetEntradaTeste:
    count =count+1
    doc = nlp(text)
    anotacao_manual = annots['entities']    
    result_model = [(item.start_char, item.end_char,item.label_) for item in doc.ents]
    #print(f'Classificação Manual:', anotacao_manual,'\n')
    #print(f'Classificação do Modelo:', result_model,'\n')
    
  
    tokens_rotulados_rotulacaomanual = [item for item in anotacao_manual if item[2] != '']
    tokens_rotulados_rotulacaoautomatica = [item for item in result_model if item[2] != '']
    
    if len(tokens_rotulados_rotulacaomanual) < 1:
        tokens_rotulados_rotulacaomanual = [("-","Sem entidade","-")]       
    if len(tokens_rotulados_rotulacaoautomatica) < 1:
        tokens_rotulados_rotulacaoautomatica = [("-","Sem entidade","-")]  
    
    print("MANUAL")
    print(tokens_rotulados_rotulacaomanual, count)
    print("AUTOMATICO")
    print(tokens_rotulados_rotulacaoautomatica, count)
    
    #for i in tokens_rotulados_rotulacaoautomatica:
     #   print(i,";","Sentença ", ";", count)      

#verifica_listas = bool(set(result_model) & set(tokens_rotulados_rotulacaomanual))

MANUAL
[(25, 38, 'NOME_BEBIDA')] 1
AUTOMATICO
[(25, 38, 'NOME_BEBIDA')] 1
MANUAL
[(7, 15, 'PRECO')] 2
AUTOMATICO
[(7, 15, 'PRECO')] 2
MANUAL
[(30, 43, 'NOME_BEBIDA'), (44, 49, 'VOLUME')] 3
AUTOMATICO
[(30, 43, 'NOME_BEBIDA'), (44, 49, 'VOLUME')] 3
MANUAL
[(8, 13, 'VOLUME')] 4
AUTOMATICO
[(8, 13, 'VOLUME')] 4
MANUAL
[(21, 26, 'GRADUACAO_ALCOOLICA')] 5
AUTOMATICO
[(21, 26, 'GRADUACAO_ALCOOLICA')] 5
MANUAL
[(42, 50, 'TIPO_MADEIRA')] 6
AUTOMATICO
[(42, 50, 'TIPO_MADEIRA')] 6
MANUAL
[('-', 'Sem entidade', '-')] 7
AUTOMATICO
[('-', 'Sem entidade', '-')] 7
MANUAL
[(10, 30, 'NOME_ORGANIZACAO')] 8
AUTOMATICO
[('-', 'Sem entidade', '-')] 8
MANUAL
[(19, 26, 'NOME_LOCAL'), (29, 38, 'NOME_LOCAL')] 9
AUTOMATICO
[(19, 26, 'NOME_LOCAL'), (29, 38, 'NOME_LOCAL')] 9
MANUAL
[(25, 34, 'NOME_BEBIDA'), (35, 41, 'TEMPO_ARMAZENAMENTO')] 10
AUTOMATICO
[(25, 34, 'NOME_BEBIDA'), (35, 41, 'TEMPO_ARMAZENAMENTO')] 10
MANUAL
[(7, 15, 'PRECO')] 11
AUTOMATICO
[(7, 15, 'PRECO')] 11
MANUAL
[(37, 53, 'RECIPIENTE_ARMAZENAM

MANUAL
[(21, 26, 'GRADUACAO_ALCOOLICA')] 70
AUTOMATICO
[(21, 24, 'GRADUACAO_ALCOOLICA')] 70
MANUAL
[(29, 37, 'TIPO_MADEIRA')] 71
AUTOMATICO
[(29, 37, 'TIPO_MADEIRA')] 71
MANUAL
[(19, 27, 'NOME_LOCAL'), (28, 30, 'NOME_LOCAL')] 72
AUTOMATICO
[(19, 27, 'NOME_LOCAL'), (28, 30, 'NOME_LOCAL')] 72
MANUAL
[(25, 42, 'NOME_BEBIDA'), (43, 48, 'VOLUME')] 73
AUTOMATICO
[(25, 42, 'NOME_BEBIDA'), (43, 48, 'VOLUME')] 73
MANUAL
[(7, 15, 'PRECO')] 74
AUTOMATICO
[(7, 15, 'PRECO')] 74
MANUAL
[(32, 49, 'NOME_BEBIDA'), (67, 78, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA'), (89, 95, 'CARACTERISTICA_SENSORIAL_AROMA'), (98, 112, 'CARACTERISTICA_SENSORIAL_AROMA')] 75
AUTOMATICO
[(32, 49, 'NOME_BEBIDA'), (67, 78, 'CARACTERISTICA_SENSORIAL_SABOR'), (89, 95, 'CARACTERISTICA_SENSORIAL_AROMA')] 75
MANUAL
[(23, 40, 'CARACTERISTICA_SENSORIAL_SABOR')] 76
AUTOMATICO
[(23, 40, 'CARACTERISTICA_SENSORIAL_SABOR')] 76
MANUAL
[(23, 32, 'NOME_LOCAL'), (37, 49, 'NOME_LOCAL')] 77
AUTOMATICO
[(23, 32, 'NOME_LOCAL'), (37, 49, 'NOME_LO

MANUAL
[(25, 31, 'NOME_BEBIDA'), (32, 45, 'CLASSIFICACAO_BEBIDA'), (46, 53, 'TEMPO_ARMAZENAMENTO'), (54, 60, 'VOLUME')] 137
AUTOMATICO
[(25, 31, 'NOME_BEBIDA'), (38, 45, 'CLASSIFICACAO_BEBIDA'), (46, 53, 'TEMPO_ARMAZENAMENTO'), (54, 57, 'TEMPO_ARMAZENAMENTO')] 137
MANUAL
[(7, 15, 'PRECO')] 138
AUTOMATICO
[(7, 15, 'PRECO')] 138
MANUAL
[(8, 13, 'VOLUME')] 139
AUTOMATICO
[(8, 13, 'VOLUME')] 139
MANUAL
[(30, 36, 'NOME_BEBIDA'), (37, 50, 'CLASSIFICACAO_BEBIDA'), (51, 58, 'TEMPO_ARMAZENAMENTO'), (59, 65, 'VOLUME')] 140
AUTOMATICO
[(30, 36, 'NOME_BEBIDA'), (43, 50, 'CLASSIFICACAO_BEBIDA'), (51, 58, 'TEMPO_ARMAZENAMENTO'), (59, 62, 'TEMPO_ARMAZENAMENTO')] 140
MANUAL
[(8, 14, 'NOME_BEBIDA'), (15, 28, 'CLASSIFICACAO_BEBIDA')] 141
AUTOMATICO
[(8, 14, 'NOME_BEBIDA'), (15, 28, 'CLASSIFICACAO_BEBIDA')] 141
MANUAL
[('-', 'Sem entidade', '-')] 142
AUTOMATICO
[('-', 'Sem entidade', '-')] 142
MANUAL
[(4, 11, 'TEMPO_ARMAZENAMENTO'), (34, 51, 'RECIPIENTE_ARMAZENAMENTO'), (55, 63, 'TIPO_MADEIRA')] 143
AUTO

MANUAL
[(25, 32, 'CLASSIFICACAO_BEBIDA'), (45, 53, 'TIPO_MADEIRA'), (54, 61, 'TEMPO_ARMAZENAMENTO'), (62, 66, 'NOME_BEBIDA'), (67, 72, 'VOLUME')] 209
AUTOMATICO
[(25, 32, 'CLASSIFICACAO_BEBIDA'), (45, 53, 'TIPO_MADEIRA'), (54, 61, 'TEMPO_ARMAZENAMENTO'), (62, 66, 'NOME_BEBIDA'), (67, 72, 'VOLUME')] 209
MANUAL
[(7, 15, 'PRECO')] 210
AUTOMATICO
[(7, 15, 'PRECO')] 210
MANUAL
[(24, 28, 'NOME_BEBIDA'), (37, 44, 'CLASSIFICACAO_BEBIDA'), (61, 67, 'TEMPO_ARMAZENAMENTO'), (71, 77, 'RECIPIENTE_ARMAZENAMENTO'), (81, 99, 'TIPO_MADEIRA'), (108, 125, 'CARACTERISTICA_SENSORIAL_COR'), (135, 140, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA')] 211
AUTOMATICO
[(24, 28, 'NOME_BEBIDA'), (37, 44, 'CLASSIFICACAO_BEBIDA'), (61, 67, 'TEMPO_ARMAZENAMENTO'), (71, 77, 'RECIPIENTE_ARMAZENAMENTO'), (81, 99, 'TIPO_MADEIRA'), (108, 125, 'CARACTERISTICA_SENSORIAL_COR'), (135, 140, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA')] 211
MANUAL
[(41, 47, 'RECIPIENTE_ARMAZENAMENTO'), (51, 59, 'TIPO_MADEIRA')] 212
AUTOMATICO
[(41, 47, '

MANUAL
[(42, 51, 'NOME_LOCAL'), (72, 78, 'NOME_LOCAL'), (79, 83, 'TEMPO')] 282
AUTOMATICO
[(42, 51, 'NOME_LOCAL'), (72, 78, 'NOME_LOCAL'), (79, 83, 'TEMPO')] 282
MANUAL
[(54, 58, 'TEMPO'), (97, 111, 'NOME_LOCAL'), (113, 125, 'NOME_LOCAL'), (127, 133, 'NOME_LOCAL')] 283
AUTOMATICO
[(54, 58, 'TEMPO'), (97, 111, 'NOME_LOCAL'), (113, 125, 'NOME_LOCAL'), (127, 133, 'NOME_LOCAL')] 283
MANUAL
[(55, 59, 'TEMPO'), (129, 138, 'NOME_LOCAL')] 284
AUTOMATICO
[(55, 59, 'TEMPO'), (129, 138, 'NOME_LOCAL')] 284
MANUAL
[(17, 37, 'NOME_BEBIDA'), (38, 43, 'CLASSIFICACAO_BEBIDA')] 285
AUTOMATICO
[(17, 37, 'NOME_BEBIDA'), (38, 43, 'CLASSIFICACAO_BEBIDA')] 285
MANUAL
[(7, 14, 'PRECO')] 286
AUTOMATICO
[(7, 14, 'PRECO')] 286
MANUAL
[(8, 14, 'VOLUME')] 287
AUTOMATICO
[(8, 14, 'VOLUME')] 287
MANUAL
[(22, 28, 'VOLUME')] 288
AUTOMATICO
[(22, 28, 'VOLUME')] 288
MANUAL
[(0, 3, 'GRADUACAO_ALCOOLICA')] 289
AUTOMATICO
[(0, 3, 'GRADUACAO_ALCOOLICA')] 289
MANUAL
[(12, 18, 'TEMPO_ARMAZENAMENTO'), (22, 28, 'RECIPIENTE_ARMA

...

MANUAL
[(10, 22, 'NOME_BEBIDA'), (23, 31, 'TIPO_MADEIRA'), (45, 52, 'CLASSIFICACAO_BEBIDA')] 3864
AUTOMATICO
[(10, 22, 'NOME_BEBIDA'), (23, 31, 'TIPO_MADEIRA'), (45, 52, 'CLASSIFICACAO_BEBIDA')] 3864
MANUAL
[(0, 6, 'NOME_LOCAL'), (9, 15, 'NOME_LOCAL')] 3865
AUTOMATICO
[(0, 6, 'NOME_LOCAL'), (9, 15, 'NOME_LOCAL')] 3865
MANUAL
[(14, 20, 'TEMPO_ARMAZENAMENTO'), (24, 52, 'TIPO_MADEIRA')] 3866
AUTOMATICO
[(14, 20, 'TEMPO_ARMAZENAMENTO'), (24, 40, 'TIPO_MADEIRA'), (43, 52, 'TIPO_MADEIRA')] 3866
MANUAL
[(23, 35, 'NOME_BEBIDA'), (36, 44, 'TIPO_MADEIRA')] 3867
AUTOMATICO
[(23, 35, 'NOME_BEBIDA'), (36, 44, 'TIPO_MADEIRA')] 3867
MANUAL
[(2, 14, 'NOME_BEBIDA'), (15, 23, 'TIPO_MADEIRA'), (37, 44, 'CLASSIFICACAO_BEBIDA')] 3868
AUTOMATICO
[(2, 14, 'NOME_BEBIDA'), (15, 23, 'TIPO_MADEIRA'), (37, 44, 'CLASSIFICACAO_BEBIDA')] 3868
MANUAL
[(12, 18, 'TEMPO_ARMAZENAMENTO'), (22, 50, 'TIPO_MADEIRA')] 3869
AUTOMATICO
[(12, 18, 'TEMPO_ARMAZENAMENTO'), (22, 38, 'TIPO_MADEIRA'), (41, 50, 'TIPO_MADEIRA')] 3869
MA

MANUAL
[('-', 'Sem entidade', '-')] 3929
AUTOMATICO
[('-', 'Sem entidade', '-')] 3929
MANUAL
[('-', 'Sem entidade', '-')] 3930
AUTOMATICO
[('-', 'Sem entidade', '-')] 3930
MANUAL
[(9, 17, 'TIPO_MADEIRA')] 3931
AUTOMATICO
[(9, 17, 'TIPO_MADEIRA')] 3931
MANUAL
[(16, 22, 'TEMPO_ARMAZENAMENTO')] 3932
AUTOMATICO
[(16, 22, 'TEMPO_ARMAZENAMENTO')] 3932
MANUAL
[(16, 19, 'GRADUACAO_ALCOOLICA')] 3933
AUTOMATICO
[(16, 19, 'GRADUACAO_ALCOOLICA')] 3933
MANUAL
[(8, 13, 'VOLUME')] 3934
AUTOMATICO
[(8, 13, 'VOLUME')] 3934
MANUAL
[(6, 10, 'CLASSIFICACAO_BEBIDA')] 3935
AUTOMATICO
[(6, 10, 'CLASSIFICACAO_BEBIDA')] 3935
MANUAL
[(8, 14, 'NOME_LOCAL'), (15, 17, 'NOME_LOCAL')] 3936
AUTOMATICO
[(8, 14, 'NOME_LOCAL'), (15, 17, 'NOME_LOCAL')] 3936
MANUAL
[(15, 25, 'TEMPO_ARMAZENAMENTO')] 3937
AUTOMATICO
[(15, 25, 'TEMPO_ARMAZENAMENTO')] 3937
MANUAL
[(12, 21, 'NOME_BEBIDA')] 3938
AUTOMATICO
[(12, 21, 'NOME_BEBIDA')] 3938
MANUAL
[(14, 18, 'TEMPO'), (20, 44, 'NOME_PESSOA'), (47, 51, 'NOME_PESSOA'), (64, 80, 'NOME_

MANUAL
[(9, 17, 'TIPO_MADEIRA'), (20, 37, 'TIPO_MADEIRA')] 4002
AUTOMATICO
[(9, 17, 'TIPO_MADEIRA')] 4002
MANUAL
[(16, 22, 'TEMPO_ARMAZENAMENTO')] 4003
AUTOMATICO
[(16, 22, 'TEMPO_ARMAZENAMENTO')] 4003
MANUAL
[(16, 19, 'GRADUACAO_ALCOOLICA')] 4004
AUTOMATICO
[(16, 19, 'GRADUACAO_ALCOOLICA')] 4004
MANUAL
[(8, 13, 'VOLUME')] 4005
AUTOMATICO
[(8, 13, 'VOLUME')] 4005
MANUAL
[(6, 10, 'CLASSIFICACAO_BEBIDA')] 4006
AUTOMATICO
[(6, 10, 'CLASSIFICACAO_BEBIDA')] 4006
MANUAL
[(8, 14, 'NOME_LOCAL'), (27, 36, 'NOME_BEBIDA'), (52, 59, 'NOME_LOCAL'), (76, 82, 'RECIPIENTE_ARMAZENAMENTO'), (86, 94, 'TIPO_MADEIRA'), (97, 115, 'TIPO_MADEIRA')] 4007
AUTOMATICO
[(8, 14, 'NOME_LOCAL'), (27, 36, 'NOME_BEBIDA'), (52, 59, 'NOME_LOCAL'), (76, 82, 'RECIPIENTE_ARMAZENAMENTO'), (86, 94, 'TIPO_MADEIRA'), (97, 115, 'NOME_PESSOA')] 4007
MANUAL
[('-', 'Sem entidade', '-')] 4008
AUTOMATICO
[('-', 'Sem entidade', '-')] 4008
MANUAL
[(9, 17, 'TIPO_MADEIRA'), (20, 37, 'TIPO_MADEIRA')] 4009
AUTOMATICO
[(9, 17, 'TIPO_MADEIRA

MANUAL
[('-', 'Sem entidade', '-')] 4072
AUTOMATICO
[('-', 'Sem entidade', '-')] 4072
MANUAL
[(26, 35, 'NOME_BEBIDA'), (36, 41, 'CLASSIFICACAO_BEBIDA'), (42, 48, 'VOLUME')] 4073
AUTOMATICO
[(26, 35, 'NOME_BEBIDA'), (36, 41, 'CLASSIFICACAO_BEBIDA'), (42, 48, 'VOLUME')] 4073
MANUAL
[(6, 12, 'NOME_LOCAL')] 4074
AUTOMATICO
[(6, 12, 'NOME_LOCAL')] 4074
MANUAL
[(8, 13, 'NOME_LOCAL')] 4075
AUTOMATICO
[(8, 13, 'NOME_LOCAL')] 4075
MANUAL
[(10, 19, 'NOME_LOCAL')] 4076
AUTOMATICO
[(10, 19, 'NOME_LOCAL')] 4076
MANUAL
[(10, 31, 'NOME_PESSOA'), (34, 54, 'NOME_ORGANIZACAO')] 4077
AUTOMATICO
[(10, 31, 'NOME_PESSOA'), (34, 54, 'NOME_ORGANIZACAO')] 4077
MANUAL
[(20, 38, 'RECIPIENTE_ARMAZENAMENTO')] 4078
AUTOMATICO
[(20, 38, 'RECIPIENTE_ARMAZENAMENTO')] 4078
MANUAL
[(19, 24, 'TEMPO_ARMAZENAMENTO')] 4079
AUTOMATICO
[(19, 24, 'TEMPO_ARMAZENAMENTO')] 4079
MANUAL
[(10, 16, 'VOLUME')] 4080
AUTOMATICO
[(10, 16, 'VOLUME')] 4080
MANUAL
[(11, 14, 'GRADUACAO_ALCOOLICA')] 4081
AUTOMATICO
[(11, 14, 'GRADUACAO_ALCOOL

MANUAL
[(25, 47, 'NOME_BEBIDA')] 4144
AUTOMATICO
[(25, 30, 'NOME_BEBIDA')] 4144
MANUAL
[(7, 16, 'VOLUME')] 4145
AUTOMATICO
[(7, 16, 'PRECO')] 4145
MANUAL
[(30, 47, 'NOME_BEBIDA'), (62, 67, 'NOME_BEBIDA'), (118, 124, 'RECIPIENTE_ARMAZENAMENTO'), (128, 135, 'TIPO_MADEIRA')] 4146
AUTOMATICO
[(30, 35, 'NOME_BEBIDA'), (62, 67, 'NOME_BEBIDA'), (93, 100, 'NOME_LOCAL'), (118, 124, 'RECIPIENTE_ARMAZENAMENTO'), (128, 135, 'TIPO_MADEIRA')] 4146
MANUAL
[(23, 30, 'CARACTERISTICA_SENSORIAL_SABOR'), (69, 76, 'NOME_LOCAL')] 4147
AUTOMATICO
[(23, 30, 'CARACTERISTICA_SENSORIAL_SABOR'), (69, 76, 'NOME_LOCAL')] 4147
MANUAL
[('-', 'Sem entidade', '-')] 4148
AUTOMATICO
[('-', 'Sem entidade', '-')] 4148
MANUAL
[('-', 'Sem entidade', '-')] 4149
AUTOMATICO
[('-', 'Sem entidade', '-')] 4149
MANUAL
[(22, 25, 'VOLUME'), (53, 55, 'TEMPO_ARMAZENAMENTO'), (68, 75, 'NOME_LOCAL')] 4150
AUTOMATICO
[(22, 25, 'VOLUME'), (53, 55, 'TEMPO_ARMAZENAMENTO'), (68, 75, 'NOME_LOCAL')] 4150
MANUAL
[(25, 36, 'CARACTERISTICA_SENSORI

with open("rotulacaomanual.txt", "w",encoding='utf-8') as arquivo:
for item in lista_rotulacaomanual:
    arquivo.write("%s\n" % str(item))
print('Done')
with open("rotulacaoautomatica.txt", "w",encoding='utf-8') as arquivo:
for item in lista_rotulacaoautomatica:
    arquivo.write("%s\n" % str(item))
print('Done')

### Métricas Precision, Recall e f-measure: 

In [15]:
# Apresenta o resultados das métricas para o modelo como um todo, e as métricas do modelo para cada categoria de entidade
calculatedBySpacy = nlp.evaluate(examples_test)
print(calculatedBySpacy)

#from spacy.scorer import Scorer
#scorer = Scorer()   
#example_scores = []
#for text, annot in dataSetEntradaTeste:
#    print(text)
#    pred = nlp.make_doc(text)
#    temp = Example.from_dict(pred, annot)
#    example_scores.append(temp)
#    scores = scorer.score(example_scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'pos_acc': None, 'morph_acc': None, 'morph_per_feat': None, 'sents_p': 1.0, 'sents_r': 1.0, 'sents_f': 1.0, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': 0.8772459196269373, 'ents_r': 0.8653768096333379, 'ents_f': 0.8712709440130773, 'ents_per_type': {'NOME_BEBIDA': {'p': 0.8, 'r': 0.7908902691511387, 'f': 0.795419052576783}, 'PRECO': {'p': 0.856140350877193, 'r': 0.9070631970260223, 'f': 0.8808664259927799}, 'VOLUME': {'p': 0.9726027397260274, 'r': 0.9281045751633987, 'f': 0.9498327759197325}, 'GRADUACAO_ALCOOLICA': {'p': 0.9667590027700831, 'r': 0.9748603351955307, 'f': 0.9707927677329623}, 'TIPO_MADEIRA': {'p': 0.9489144316730523, 'r': 0.9138991389913899, 'f': 0.931077694235589}, 'TEMPO_ARMAZENAMENTO': {'p': 0.900497512437811, 'r': 0.9329896907216495, 'f': 0.9164556962025316}, 'NOME_ORGANIZACAO': {'p': 0.8480565371024735, 'r': 0.75, 'f': 0.7960199004975125}, 'NOME_LOCAL': {'p': 0.9333849728892

### Avaliação do Modelo pela Métrica Acurácia

In [ ]:
# dictionary which will be populated with the entities and result information
entity_evaluation = {}
# helper function to udpate the entity_evaluation dictionary
def update_results(entity, metric):
    if entity not in entity_evaluation:
        entity_evaluation[entity] = {"correct": 0, "total": 0}  
    entity_evaluation[entity][metric] += 1
    
# same as before, see if entities from test set match what spaCy currently predicts
example_scores = []
for data in dataSetEntradaTeste:
    sentence = data[0]
    entities = data[1]["entities"]
    for entity in entities:
        doc = nlp(sentence)
        correct_text = sentence[entity[0]:entity[1]]
        for ent in doc.ents:
            if ent.label_ == entity[2] and ent.text == correct_text:
                update_results(ent.label_, "correct")
                break
        update_results(entity[2], "total")

print("Dados corretamente classificados versus total de dados corretos:",'\n', entity_evaluation)

In [ ]:
print("Calculo Acuracia por Categoria e Acuracia Total")
sum_total = 0
sum_correct = 0
sum_incorretos_total = 0
accuracy_by_category = {}

for entity in entity_evaluation:
    if entity != '':
        total = entity_evaluation[entity]["total"]
        correct = entity_evaluation[entity]["correct"]
        rotulados_incorretamente = total-correct
        sum_total += total
        sum_correct += correct
        sum_incorretos_total += rotulados_incorretamente
        percentual_acertos_modelo_por_categoria = correct/total
        #total_acerto_por_categoria = sum_accuracy_total
        print("{} --> {:.2f}%".format(entity, percentual_acertos_modelo_por_categoria))
        
        accuracy_by_category[entity] = {str(percentual_acertos_modelo_por_categoria)}#para salvar no arquivo .txt com o percentual
sum_accuracy = sum_correct / sum_total 
sum_accuracy_total = str(sum_accuracy) #para salvar no arquivo .txt com o percentual

print("Acuracia Total (todas as categorias): {:.2f}%".format(sum_accuracy))
print('Total de entidades Rotulados Manualmente (referencia):', sum_total)
print('Total de entidades Rotuladas Corretamente pelo modelo:', sum_correct)
print('Total de entidades Rotuladas Incorretamente pelo modelo:', sum_incorretos_total)

### Salvados os resultados em um arquivo de texto

In [ ]:
#Salvando informações relevantes em um arquivo.txt
nome_arquivo = '  128epochs64batchs.txt'
with open ('./ResultadosDosTestes/'+nome_arquivo,"w", encoding='utf-8')  as output:
    quantidadeDados = str(len(dataSetEntradaTeste))
    output.write('Tamanho do dataset de teste: '+quantidadeDados)
    output.write('\n') 
    
    output.write('\n'+"|CONFIGURAÇÕES UTILIZADAS PARA TREINAR O MODELO:|"+'\n')
    epochsSize_String = str(epochsSize)
    batchSize_String = str(batchSize)
    drop_String = drop_size
    output.write('Total de épocas: '+epochsSize_String+"\n")
    output.write('Total de batches: '+batchSize_String+"\n")
    output.write('Tamanho do Drop:')
    output.write(str(drop_size))
    output.write('\n') 
                 
    #Informações sobre treinamento do modelo
    output.writelines('\n'+"|INFORMAÇÕES SOBRE TREINAMENTO DO MODELO:|"+'\n')
    output.writelines('Perda acumulada do modelo (soma de todas as perdas): '+str(losses))
    output.write('\n')
    output.writelines('Tempo gasto para treinar o modelo em segundos: '+str(tempoParaTreinarModelo))
    output.write('\n') 
    
    #Métricas geradas pelo método 'evaluat', disponibilizado pelo próprio spaCy.
    output.writelines('\n'+"|RESULTADOS DO MODELO PARA OS DADOS DO DATASET DE TESTE|")    
    output.writelines('\n'+"|Abaixo temos Precison, Recall, F-measure e Acuracia Geral:|"+'\n')
    ents_p = "Precisão Total: "+str(round(calculatedBySpacy.get("ents_p"),4))+'\n'
    ents_r = "Revocação Total: "+str(round(calculatedBySpacy.get("ents_r"),4))+'\n'
    ents_f = "F-measure Total: "+str(round(calculatedBySpacy.get("ents_f"),4))+'\n'
    ents_a = "Acurácia Total: "+str(round(sum_accuracy,4))+'\n'
    total_dados_com_rotulos = "Total de entidades rotuladas existentes no dataset de teste: "+str(round(sum_total,2))+'\n'
    total_dados_rotulados_corretamento = "Total de entidades classificadas corretamente pelo modelo: "+str(round(sum_correct,2))+'\n'
    total_dados_rotulados_incorretamentos = "Total de entidades classificadas incorretamente pelo modelo: "+str(round(sum_incorretos_total,2))+'\n'
    
    output.write(ents_p),output.write(ents_r),output.write(ents_f),output.write(ents_a)
    output.write(total_dados_com_rotulos),output.write(total_dados_rotulados_corretamento)
    output.write(total_dados_rotulados_incorretamentos)
    #output.writelines(';'.join(str(x) for x in (token_acc,token_p)))
    
    output.writelines('\n'+"|Abaixo temos Precision, Recall e F-measure para cada categoria:|"+'\n')
    ents_per_type = calculatedBySpacy.get("ents_per_type")
    for key in ents_per_type:
        ents_por_tipo = str(key)+str(ents_per_type[key])+'\n'
        output.write(ents_por_tipo) 
   
    #Acuracia do modelo por categoria.
    output.writelines('\n'+"|Abaixo temos a Acurácia para cada categoria:|"+'\n')
    for key in accuracy_by_category:
        entity_evaluation_accuracy = str(key)+str(accuracy_by_category[key])+'\n'
        output.write(entity_evaluation_accuracy)
    
    #Total de acertos e erros do modelo por categoria.    
    output.writelines('\n'+"|Abaixo temos o total de acertos do modelo para cada categoria Versus o total que ele deveria classificar:|"+'\n')
    for key in entity_evaluation:
        if key != '':
            acertos_erros = str(key)+str(entity_evaluation[key])+'\n'
            output.write(acertos_erros)
            
    #Parametros digitados no terminal pelo usuario
    #output.write('\n'+"|ABAIXO TEMOS OS PARAMETROS INFORMADOS NO TEMRINAL:|"+'\n')
    #output.writelines(';'.join(str(x) for x in (primeiraEntradaTerminal,segundaEntradaTerminal,terceiraEntradaTerminal,quartaEntradaTerminal,quintaEntradaTerminal)))

## Salvando o Modelo

In [ ]:
# Save the  model to directory
nome_modelo = '128epochs64batchs'
from pathlib import Path
output_dir=Path('./ResultadosDosTestes/'+nome_modelo)
nlp.to_disk(output_dir)
print("Modelo salvo!", output_dir)

## Importando o Modelo

In [ ]:
# Load the saved model and predict

#print("Loading from", output_dir)
#nlp_updated = spacy.load(output_dir)
#doc = nlp_updated("Eu gostei da Cachaça Artesanal criada em Minas Gerais" )
#print("Entidades", [(ent.text, ent.label_) for ent in doc.ents])

In [157]:
lista1 = []
lista2 = []

with open('lista1.txt', 'r') as f:
    for l in f:
        lista1.append(l.rstrip())

with open('lista2.txt', 'r') as f:
    for l in f:
        lista2.append(l.rstrip())

In [161]:
lista_acerto_modelo = []
for i in lista2:
    for j in lista1:
        if i == j :
            lista_acerto_modelo.append(j)